### 作業目的: 熟練Pytorch Dataset與DataLoader進行資料讀取

本此作業主要會使用[IMDB](http://ai.stanford.edu/~amaas/data/sentiment/)資料集利用Pytorch的Dataset與DataLoader進行
客製化資料讀取。
下載後的資料有分成train與test，因為這份作業目的在讀取資料，所以我們取用train部分來進行練習。
(請同學先行至IMDB下載資料)

### 載入套件

In [1]:
# Import torch and other required modules
import glob
import torch
import re
import nltk
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_svmlight_file
from nltk.corpus import stopwords

nltk.download('stopwords') #下載stopwords
nltk.download('punkt') #下載word_tokenize需要的corpus

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\program\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\program\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### 探索資料與資料前處理
在train資料中，有分成pos(positive)與neg(negative)，分別為正評價與負評價，此評價即為label。

In [2]:
import string

In [15]:
# 讀取字典，這份字典為review內所有出現的字詞
with open('./aclImdb/imdb.vocab', 'rt', encoding = 'utf8') as f:
    vocab = f.read().splitlines()

# 以nltk stopwords移除贅字，過多的贅字無法提供有用的訊息，也可能影響模型的訓練
print(f"vocab length before removing stopwords: {len(vocab)}")
vocab = [v for v in vocab if v not in (stopwords.words('english') + list(string.punctuation))]
print(f"vocab length after removing stopwords: {len(vocab)}")

# 將字典轉換成dictionary
vocab_dic = {k:v for k, v in zip(vocab, range(len(vocab)))}
vocab_dic

vocab length before removing stopwords: 89527
vocab length after removing stopwords: 89354


{'movie': 0,
 'film': 1,
 'one': 2,
 'like': 3,
 'good': 4,
 'even': 5,
 'time': 6,
 'would': 7,
 'story': 8,
 'really': 9,
 'see': 10,
 'well': 11,
 'much': 12,
 'get': 13,
 'people': 14,
 'bad': 15,
 'also': 16,
 'great': 17,
 'first': 18,
 'made': 19,
 'way': 20,
 'make': 21,
 'could': 22,
 'movies': 23,
 'think': 24,
 'characters': 25,
 'character': 26,
 'watch': 27,
 'films': 28,
 'two': 29,
 'many': 30,
 'seen': 31,
 'acting': 32,
 'never': 33,
 'plot': 34,
 'little': 35,
 'love': 36,
 'best': 37,
 'life': 38,
 'show': 39,
 'know': 40,
 'ever': 41,
 'better': 42,
 'man': 43,
 'still': 44,
 'end': 45,
 'say': 46,
 'scene': 47,
 'scenes': 48,
 'go': 49,
 'something': 50,
 'back': 51,
 "i'm": 52,
 'watching': 53,
 'real': 54,
 'though': 55,
 'thing': 56,
 'years': 57,
 'actors': 58,
 'director': 59,
 'another': 60,
 'nothing': 61,
 'new': 62,
 'funny': 63,
 'actually': 64,
 'work': 65,
 'makes': 66,
 'find': 67,
 'look': 68,
 'old': 69,
 'going': 70,
 'lot': 71,
 'part': 72,
 'every

In [30]:
# 將資料打包成(x, y)配對，其中x為review的檔案路徑，y為正評(1)或負評(0)
# 這裡將x以檔案路徑代表的原因是讓同學練習不一次將資料全讀取進來，若電腦記憶體夠大(所有資料檔案沒有很大)
# 可以將資料全一次讀取，可以減少在訓練時I/O時間，增加訓練速度

review_pairs = list(map(lambda x: (x, 1), glob.glob('aclImdb/train/pos/*'))) \
+ list(map(lambda x: (x, 0), glob.glob('aclImdb/train/neg/*')))
print(review_pairs[:2])
print(f"Total reviews: {len(review_pairs)}")

[('aclImdb/train/pos\\0_9.txt', 1), ('aclImdb/train/pos\\10000_8.txt', 1)]
Total reviews: 25000


### 建立Dataset與DataLoader讀取資料
這裡我們會需要兩個helper functions，其中一個是讀取資料與清洗資料的函式(load_review)，另外一個是生成詞向量BoW的函式
(generate_bow)

In [62]:
def load_review(review_path):
    
    with open(review_path, 'rt', encoding = 'utf8') as f:
        review = f.read()
        
    #移除non-alphabet符號、贅字與tokenize
    review = re.sub(r'\W+', ' ', review)
    review = nltk.tokenize.word_tokenize(review)
    review = [w for w in review if w not in stopwords.words('english')]
    
    return review

In [65]:
def generate_bow(review, vocab_dic):
    bag_vector = np.zeros(len(vocab_dic))
    for word in review:
        if vocab_dic.get(word):
            bag_vector[vocab_dic.get(word)] += 1
            
    return bag_vector

In [66]:
class dataset(Dataset):
    '''custom dataset to load reviews and labels
    Parameters
    ----------
    data_pairs: list
        directory of all review-label pairs
    vocab: list
        list of vocabularies
    '''
    def __init__(self, data_dirs, vocab):
        self.data_dirs = data_dirs
        self.vocab = vocab

    def __len__(self):
        return len(self.data_dirs)

    def __getitem__(self, idx):
        review_path = self.data_dirs[idx][0]
        review_direction = self.data_dirs[idx][1]
        
        review = load_review(review_path)
        bag_vector = generate_bow(review, self.vocab)
        return (bag_vector, review_direction)

In [69]:
# 建立客製化dataset
custom_dst = dataset(review_pairs, vocab_dic)
custom_dst[10]

(array([0., 2., 1., ..., 0., 0., 0.]), 1)

In [73]:
# 建立dataloader
custom_dataloader = DataLoader(custom_dst, batch_size=4, shuffle=True)
next(iter(custom_dataloader))

[tensor([[0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 3., 0.,  ..., 0., 0., 0.]], dtype=torch.float64),
 tensor([1, 1, 0, 0])]